### 外部に用意されたデータを読み込む

いつも通りD-Wave Ocean SDKの準備を終えたら本日も行ってみましょう！
本日はさらに実用的なステップを踏んでいきます。

In [ ]:
pip install dwave-ocean-sdk

今後も利用されることがあると思いますので、ローカルファイルをGoogle colab上にアップロードして使う方法を紹介します。
まず次のようにライブラリを読み込みます。

In [ ]:
from google.colab import files

この後以下のコードを実行するとファイルをアップロードすることができます。
講義時に配布したリンクからダウンロードしたファイルをGoogle colabにアップロードしてください。

In [ ]:
files.upload()

Saving QA4Uafter.csv to QA4Uafter (4).csv


{'QA4Uafter.csv': b'3.0,3.0,3.0,3.0\n2.0,2.0,3.0,3.0\n4.0,3.0,4.0,5.0\n4.0,3.0,3.0,4.0\n3.0,2.0,3.0,6.0\n3.0,3.0,3.0,5.0\n4.0,1.0,3.0,3.0\n3.0,2.0,2.0,2.0\n4.0,3.0,3.0,4.0\n2.0,3.0,3.0,3.0\n2.0,3.0,4.0,4.0\n4.0,2.0,2.0,4.0\n4.0,3.0,5.0,5.0\n2.0,1.0,2.0,4.0\n3.0,3.0,3.0,4.0\n2.0,2.0,3.0,2.0\n2.0,2.0,3.0,4.0\n2.0,2.0,3.0,3.0\n2.0,1.0,1.0,3.0\n2.0,1.0,3.0,1.0\n4.0,2.0,4.0,4.0\n2.0,1.0,3.0,5.0\n4.0,1.0,5.0,2.0\n2.0,2.0,3.0,1.0\n4.0,1.0,2.0,7.0\n4.0,2.0,3.0,4.0\n4.0,3.0,3.0,5.0\n4.0,1.0,3.0,3.0\n3.0,2.0,3.0,2.0\n4.0,3.0,3.0,5.0\n3.0,2.0,1.0,2.0\n2.0,1.0,3.0,4.0\n2.0,2.0,3.0,5.0\n4.0,2.0,2.0,4.0\n3.0,2.0,3.0,3.0\n4.0,2.0,2.0,1.0\n3.0,3.0,2.0,1.0\n2.0,2.0,3.0,5.0\n4.0,1.0,2.0,1.0\n4.0,3.0,3.0,3.0\n3.0,3.0,3.0,5.0\n3.0,1.0,3.0,5.0\n4.0,3.0,3.0,1.0\n1.0,2.0,1.0,4.0\n2.0,1.0,1.0,3.0\n4.0,1.0,3.0,1.0\n4.0,3.0,3.0,3.0\n4.0,3.0,3.0,4.0\n1.0,1.0,3.0,2.0\n2.0,1.0,2.0,1.0\n2.0,3.0,2.0,4.0\n4.0,2.0,2.0,1.0\n4.0,2.0,3.0,3.0\n2.0,1.0,2.0,5.0\n4.0,3.0,3.0,1.0\n4.0,1.0,2.0,4.0\n2.0,2.0,2.0,2.0\n4.0,3.0,3.0

今回は本イベントに参加した皆さんから演習パートに参加したいガチ勢の方々のアンケート結果を利用してグループ分けをしたいと思います。
先ほどアップロードしたファイルをpandasというライブラリで利用します。

In [ ]:
import pandas as pd
df = pd.read_csv('QA4Uafter.csv',header=None)

ここでdfはデータフレームと呼ばれる形式になっています。
利用の仕方はpandasを検索すると様々な方法があることがわかることとももいます。
特に今回私たちが必要なのはcsvに入った数値ですので、次のように操作します。

In [ ]:
import numpy as np
V = df.values.astype(np.int8)
print(V)

Vに読み込んだデータを行列として格納しました。
どのようなサイズのデータかを調べるためには.shapeを利用しましょう。

In [ ]:
N,M = V.shape
print(N,M)

246 4


これによると241人分のデータで、4項目のデータが格納されているということがわかります。
実際それぞれ4項目、つまり4列の数値が並んでいると思います。
これは事前に取ったアンケートの結果の数値となっています。

このアンケートの結果を利用してグループ分けを行いたいと思います。

### どんなコスト関数にしようか

それぞれの人にどんなデータが割り当てられているか調べてみましょう。
V[i,:]とするとiで指定された人のデータが出てきます。

In [ ]:
print(V[0,:])
print(V[1,:])

例えばこれは1番目の人のデータと、2番目の人のデータです。
この番号は、キャラクターや役割、思考傾向についてのアンケートで選んでもらった番号です。
同じ番号を選んでいれば似たようなキャラクターや役割を持っていることになります。
異なる番号であれば、ちょっと違うキャラクターや役割をもっているということがわかります。

そこでできるだけ異なる人同士が同じグループに所属するようにできたら良いなと考えます。
そのためにはこれらの異なるデータが似ているか似ていないかを示すように計算をする必要があります。
そのためにはどうしたら良いでしょうか。
3という数字と2という数字は異なります。数字が近いからと言って、特性が似ているとは限りません。
あくまで同じかどうかが重要です。
それが計算できるように加工しましょう。

### one-hot表示

ここで新しい情報の表現方法を紹介します。
one-hot表示というものです。複数の要素があり、どれが利用されているかを0と1で示すものです。
例えば3は$(0,0,1,0,\ldots)$と3番目に1が立ち、2は$(0,1,0,0,\ldots)$と2番目に1が立ちます。
そのようなベクトルを用意すると、内積という計算で**同じかどうか**がわかります。
内積という計算は、各成分をかけて足し上げるということになっています。
3と2の場合は、
\begin{equation}
0 \times 0 + 0 \times 1 + 1 \times 0 + 0 \times 0 +\ldots = 0
\end{equation}
となります。3と3であれば
\begin{equation}
0 \times 0 + 0 \times 0 + 1 \times 1 + 0 \times 0 +\ldots = 1
\end{equation}
となります。これで同じかどうかを示すことができそうです。

今回のアンケートでは、4つの項目があり、それぞれ4,3,5,7個の選択肢から答えを選んでもらいました。
それぞれ4つのうち1個が、3つのうち1個が、5つのうち1個が、７つのうち1個が1を取り、それ以外全て0となっているようなデータに加工すると都合がよさそうです。




その加工のために次のような関数を作ります。
実は関数というのは自分で作り出すことができます。def 関数名（引数名）:として、その中でどんな計算を実行するのか、自由に書くことができます。
最後に結果を返すreturnで計算結果を出すようにもできます。

In [ ]:
def onehot(data):
  z = np.zeros(4+3+5+7)
  z[data[0]-1] = 1
  z[4+data[1]-1] = 1
  z[4+3+data[2]-1] = 1
  z[4+3+5+data[3]-1] = 1
  return z

この関数を利用すると、加工されたデータのベクトルが出力されますので、一個ずつデータを加工していきましょう。

In [ ]:
M = 4+3+5+7
Z = np.zeros(N*M).reshape(N,M)

for i in range(N):
  Z[i,:] = onehot(V[i,:])

これで加工終了です。試しに1番目と2番目のデータがどれだけ似ているかをみてみましょう。

In [ ]:
np.dot(Z[0,:],Z[1,:])

2.0

似ている度合いは2ということになりました。確かに同じ答えは2つありましたから、リーズナブルな結果になりました。

これを数式で示して、$Z_{i,m}$として、i番目の人のもつ$m$番目の特徴を表すということで今後利用していきましょう。
あとは量子アニーリングマシンで最適化をする準備を整えます。

### 2つの添字は大丈夫ですか？

組合せ最適化問題では、どれを選ぶかという問題が基本となります。
例えば荷物の選択の問題はその一種でした。
その時には$x_i$という文字式を利用して、
どの荷物($i$番目）を持つか持たないか（$x_i= 0,1$）を決めるとかでしたね。

その荷物は誰が持つのか。人の選択もできるようになれば、非常に便利で考え方の幅が広がるかと思います。

そのためには添字が$2$つあれば良いですね。
$x_{ik}$として、誰が（$i$番目)、どれを（$k番目$）を持つか持たないかを決めることにしましょう。
誰が座標と、どれを座標の$i,k$があると考えても良いかもしれません。


|$x_{ik}$  | Aさん | Bさん | Cさん | Dさん | Eさん |
|--:|------|------|------|------|------|
| 荷物A |      | 1    |      |      |      |
| 荷物B | 1    |      |      |      |      |
| 荷物C |      |      | 1    |      |      |
| 荷物D |      |      |      |      | 1    |
| 荷物E |      |      |      | 1    |      |

それではこの考え方を利用して、
誰がどのグループに所属するのか、ということを決めることにしましょう。

$x_{ik}$により、$i$番目の人が$k$番目のグループに入るという意味とします。

|$x_{ik}$  | Aさん | Bさん | Cさん | Dさん | Eさん |
|--:|------|------|------|------|------|
| グループ１ |      | 1    |      |      |      |
| グループ2 | 1    |      |      |      |      |
| グループ3 |      |      | 1    |      |      |
| グループ4 |      |      |      |      | 1    |
| グループ5 |      |      |      | 1    |      |


全員どこかのグループに所属します。
そうするとグループを全て足し算すると、1つのグループに所属していることから
\begin{equation}
\sum_{k=1}^{K} x_{ik} = 1
\end{equation}
となります。これを罰金法で書けば、
\begin{equation}
\lambda \sum_{i=1}^{N} \left( \sum_{k=1}^{K} x_{ik} - 1 \right)^2
\end{equation}
という項をコスト関数に追加すれば、グループ分けができます。


### コスト関数を設計
さて本題となる異なる特性を持つ人を探すということを考えてみましょう。
同じグループに所属する人はできるだけ異なる特性になって欲しいというお願い事があります。
$i$番目の人が持つ$m$番目の特性は$Z_{i,m}$として書くことができます。
他の$j$番目の人は同じように特性を$Z_{j,m}$としてもっています。
これらの内積を計算するには、
\begin{equation}
\sum_{m} Z_{i,m}Z_{j,m}
\end{equation}
として内積を計算します。
この$i$番目と$j$番目の人が同じ$k$番目のグループにいるかどうかを調べてみると、$x_{i,k}x_{j,k}$で調べることができます。両方が$1$となっている場合のみ$x_{i,k}x_{j,k}=1$となります。
そうなると$i$と$j$について合計すれば（ただし$i=j$は除く）、$k$番目のグループにいる人を全て勘案することができます。
ここで先に$\sum_{m} Z_{i,m}Z_{j,m}$をかけておけば、$k$番目のグループの中で、似ている度合いを合計することができます。
これを最小化すれば、できるだけ似ていないもの同士が同じグループに含まれることになります。

\begin{equation}
E({\bf x}) = \sum_{k=1}^{K} \sum_{i \neq j} \left( \sum_{m}Z_{i,m}Z_{j,m} \right) x_{i,k}x_{j,k} + \lambda_1 \sum_{i=1}^{N} \left( \sum_{k=1}^{K} x_{ik} - 1 \right)^2
\end{equation}
定式化ができました！

それではいざ最適化といきましょう。
pyQUBOで$2$つの添え字がある場合も書くことができます。

In [ ]:
from pyqubo import Array
K = 41
# バイナリ変数
x = Array.create(name='x', shape=(N,K), vartype='BINARY')

前回同様pyQUBOからArrayというクラスを呼び出します。
Array.create()で数式で使われる文字式として、xを用意します。
そのxはshape=(N,K)として、2つの添え字がどこまで進めるのかを示します。$N$は人の人数、$K$はグループの数ですね。
'BINARY'で2値の$0$または$1$をとる場合を考えます。
これで$x_{ik}$の完成です。

これで数式のままコスト関数を書くことができます。

ただ残念ながらメモリ不足等によりマシンに負担がかかるほど巨大な問題を相手にしていますので、極力マシンへの負担を減らす形で実装してみましょう。

先に$\sum_{m} Z_{im}Z_{jm}$の計算をしておきましょう。
行列$A$、$B$に対して$\sum_{j}A_{ij}B_{jk}$という計算を実行したい場合はnp.dot(A,B)とします。
足し算のされる添え字が後、前の場合はそれで良いですが、前、前の場合、つまり$\sum_{j}A_{ji}B_{jk}$の場合はnp.dot(A.T,B)とします。
後、後の場合、つまり$\sum_{j}A_{ij}B_{kj}$の場合はnp.dot(A,B.T)とします。
これを利用すると


In [ ]:
Zmat = np.dot(Z,Z.T)

としてあらかじめ$\sum_{m} Z_{im}Z_{jm}$を計算しておくことができます。

まずは罰金法による第二項は、展開をしておくとQUBO行列のプログラムの仕方が見えてきます。
\begin{equation}
\lambda_1 \sum_{i=1}^{N} \left( \sum_{k=1}^{K} x_{ik} - 1 \right)^2 = \lambda_1 \sum_{i=1}^{N} \left( \sum_{k=1}^{K} x_{ik} - 1 \right)\left( \sum_{l=1}^{K} x_{il} - 1 \right) = 
\lambda_1 \sum_{i=1}^{N}\sum_{k=1}^{K} \sum_{l=1}^{K} x_{ik} x_{il}  - 2 \lambda_1 \sum_{i=1}^{N}\sum_{k=1}^{K} x_{ik} + \lambda_1 N
\end{equation}
愚直にfor文でこの数式通りのQUBOを作りましょう。
メモリを多く利用しないようにするために、行列(np.array)形式ではなく
dict形式で直接的に入れていきます。


In [ ]:
lam1 = 20.0
K = 41

QUBO = {}
for i in range(N):
  for k in range(K):
    for l in range(K):
      QUBO[(i+k*N,i+l*N)] = lam1
      if k == l:
        QUBO[(i+k*N,i+l*N)] = QUBO[(i+k*N,i+l*N)] - 2*lam1

対角項の分は、if文による条件で追加して足し算する書き方にしています。
k == lとありますが、添え字が等しいときは$-2\lambda$するということを書いています。
数式のままですね。

ここで$x_{i,k}$にかかる係数をi+k*Nとしています。
これは二つの添字がある場合には、i=1,2,...,Nとk=1,2,...,Kを1つにまとめた表記をするというテクニックを利用します。1,2,...,N,N+1,N+2,...,2N,2N+1,2N+2,...と続けてしまい、
最初の1,2,...,Nはk=1用、N+1,N+2,...,2Nはk=2用として使えば区別できますよね。
そのための表記方法がi+k*Nです。

$K=41$という数字はメンバーの数が$246$人で、それを$6$で割ると$41$グループほどできるということからです。


これに追加して第一項も考慮してみましょう。dict形式では、存在しない要素（key）にアクセスすると、そういうものはないよ、と怒られるので、すでにあるかどうかをチェックする必要があります。
シグマ記号とfor文は対応するとしても、そこだけ気遣った表記にする必要があります。

In [ ]:
for k in range(K):
  for i in range(N):
    for j in range(N):
      if i != j:
        if (i+k*N,j+k*N) not in QUBO.keys():
          QUBO[(i+k*N,j+k*N)] = Zmat[i,j]
        else:
          QUBO[(i+k*N,j+k*N)] = QUBO[(i+k*N,j+k*N)] + Zmat[i,j]

これでQUBO行列をdict形式で用意することに成功しました。
量子アニーリングマシンに投入しましょう。
いつものようにAPI tokenを用意して準備を始めます。

In [ ]:
token = '**'
endpoint = 'https://cloud.dwavesys.com/sapi/'

今回の問題は246人、41グループですから添字の数を数えると246*41=10086個の0と1の組み合わせを考えなければならないという問題です。
D-Wave Advantageも最大で5000程度の変数の問題しか扱えません。
ハイブリッドソルバーを利用しましょう。

前回紹介したようにハイブリッドソルバーを利用するとしても、そんな面倒なことはなくソルバーとして次のように指定するだけです。

In [ ]:
from dwave.system import LeapHybridSampler
sampler = LeapHybridSampler(solver='hybrid_binary_quadratic_model_version2', token=token, endpoint=endpoint)

ソルバーを指定したら、あとはQUBO行列を投入するだけです。

In [ ]:
sampleset = sampler.sample_qubo(QUBO)

多少時間がかかりますが、巨大な問題を送付して、解いているので仕方がありません。
結果の表示は次のとおりです。

In [ ]:
print(sampleset.record)

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0

この結果を見やすい形に整形しましょう。
sampleset.record[0][0]をreshapeします。

In [ ]:
ans = sampleset.record[0][0].reshape(N,K)

これで縦に人ごと、横にグループごとに並んだ数値表を得ることができます。
ちゃんと制約条件を守っているかどうかを調べるためには、縦に足し算をしましょう。

In [ ]:
np.sum(ans,axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

それぞれのグループに何人いるのかを調べるためには、
np.sum(ans,axis=0)で計算できます。

In [ ]:
np.sum(ans,axis=0)

array([6, 5, 7, 6, 7, 6, 6, 6, 6, 7, 7, 6, 6, 4, 6, 5, 6, 4, 5, 6, 6, 6,
       5, 6, 7, 6, 7, 6, 5, 6, 6, 6, 6, 6, 6, 7, 6, 7, 6, 6, 7])

多少人数のばらつきがあることがわかります。
それもそのはずで、6人ずつのグループにしましょうという制約条件を課してはいませんでした。
厳密にグループの人数を縛る必要もない場合には、こうした結果でも良いかもしれません。
それではグループの人数をきっちり6人にするという場合には、次のような制約条件を課しましょう。
ある$k$番目のグループに入っている人数を合計すると$6$人になるようにせよ、ということですから、
\begin{equation}
\sum_{i=1}^N x_{ik} = 6
\end{equation}
です。
罰金法で実行するには、
\begin{equation}
\lambda_2 \sum_{k=1}^K \left( \sum_{i=1}^N x_{ik} - 6 \right)^2
\end{equation}
としましょう。


これも同様に展開すれば、
\begin{equation}
\lambda_2 \sum_{k=1}^K \left( \sum_{i=1}^N x_{ik} - 6 \right)^2 = \lambda_2 \sum_{k=1}^K \left( \sum_{i=1}^N x_{ik} - 6 \right)\left( \sum_{j=1}^N x_{jk} - 6 \right)
\end{equation}
\begin{equation}
= 
\lambda_2 \sum_{k=1}^K \sum_{i=1}^N \sum_{j=1}^N x_{ik}x_{jk} - 12\lambda_2 \sum_{i=1}^N x_{ik} + 36 \lambda_2 K
\end{equation}

QUBO行列に新しい項を追加します。

In [ ]:
lam2 = 3.0
for k in range(K):
  for i in range(N):
    for j in range(N):
      QUBO[(i+k*N,j+k*N)] = QUBO[(i+k*N,j+k*N)] + lam2
      if i == j:
        QUBO[(i+k*N,j+k*N)] = QUBO[(i+k*N,j+k*N)] - 2*6*lam2
      

この変更されたQUBO行列を再び量子アニーリングマシンに投入しましょう。

In [ ]:
sampleset = sampler.sample_qubo(QUBO)
print(sampleset.record)
ans = sampleset.record[0][0].reshape(N,K)

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

結果を取り出して加工するところは同じ手順です。

In [ ]:
np.sum(ans,axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

きっちり全てどこかのグループに所属しながら、


In [ ]:
np.sum(ans,axis=0)

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])

全部のグループで、6人ずつが参加しています。
どのグループに各人所属しているのかを調べるためには、次のようなコードで調べることができます。

In [ ]:
group_num = np.zeros(N)
for i in range(N):
  temp = np.where(ans[i,:]==1)[0][0]
  group_num[i] = temp
  print("{0} : {1}".format(i,temp))

これで何番目の人がどのグループに所属しているかを示す行列を得ることができました。
その結果から各グループに所属している人の番号が分かります。

In [ ]:
for k in range(K):
  itemp = np.where(group_num == k)[0]
  print("{0}th group : {1}".format(k+1,itemp+1))

この結果を各グループごとにBCCでメール送信します。
ご自身のグループ番号ごとにslackでチャンネルを作って交流をし始めましょう。